# Métodos de aprendizaje automático 2017 laboratorio 1

### En esta primera etapa se realizará un experimento para mostrar cómo se puede aplicar erroneamente la selección de características.

Relizamos la importación de los módulos necesarios.

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn import feature_selection
from sklearn.pipeline import Pipeline
import random
import numpy as np

Generamos una matriz con 10000 características aleatorias y una etiqueta a predecir también con valores aleatorios.

In [2]:
random.seed(0)
X = np.random.randint(0,10,size=[100,10000])
y = np.random.randint(0,2,size=100)

Aplicamos feature selection con el método de chi cuadrado.

In [3]:
fs = feature_selection.SelectKBest(feature_selection.chi2, k=20)
X_fs = fs.fit_transform(X, y)

Generamos un clasificador. En este caso una regresión logística.

In [4]:
clf = LogisticRegression()

Aplicamos validación cruzada para ver los resultados obtenidos.

In [5]:
scores = cross_val_score(clf, X_fs, y, cv=5, scoring='accuracy')
print scores

[ 0.9047619   0.95        0.75        0.85        0.68421053]


Explicar por qué si los números son generados al azar se obtienen buenos resultados. ¿Cómo solucionaría este problema?
Resolver el problema utilizando pipelines de sklearn

Con el método de chi cuadrado se obtienen las 20 features que "clasifican mejor" a las instancias.
Por más que se hayan generado todos los datos (valores por cada feature para cada instancia, y las respectivas clasificaciones para cada instancia) de forma aleatoria, y al tener inicialmente una cantidad considerablemente grande de features (10000), probablemente hayan features que casualmente ayuden a clasificar las instancias mejor que otros.
Entonces, si a partir de un conjunto X nos quedamos con las 20 mejores features según chi cuadrado, y después aplicamos validación cruzada con un clasificador de regresión logística (LogisticRgression()) usando este X para luego definir los conjuntos de entrenamiento y de prueba, es esperable que dé buenos resultados para cada iteración de la validación cruzada, ya que tanto para el conjunto de entrenamiento como para el de prueba se usan los 20 features que mejor clasifican a ambos.
Para solucionar el problema, la elección de los 20 features con chi cuadrado debería realizarse únicamente con los conjuntos de entrenamiento Xi de cada iteración, para luego evaluar contra un subconjunto de entrenamiento que no participó en la elección de las 20 mejores features. De esta forma obtenemos resultados más realistas de la precisión del clasificador.

In [6]:
estimator = Pipeline([("fs", fs),
                      ("clf",clf)])
scores = cross_val_score(estimator, X, y, cv=5, scoring='accuracy')
print scores

[ 0.57142857  0.45        0.45        0.45        0.47368421]


### Para la siguiente parte trabajaremos con un conjunto de datos de textos. Para ello los descargamos.
Una descripción del set de datos: http://qwone.com/~jason/20Newsgroups/

In [7]:
from sklearn.datasets import fetch_20newsgroups
train = fetch_20newsgroups(subset='train', categories=['talk.politics.guns', 'soc.religion.christian', 'comp.graphics', 'sci.med'], shuffle=True, random_state=42)
test = fetch_20newsgroups(subset='test', categories=['talk.politics.guns', 'soc.religion.christian', 'comp.graphics', 'sci.med'], shuffle=True, random_state=42)

Mostrar un ejemplo de entrenamiento de cada una de las categorías.

In [8]:
from pprint import pprint
import pandas as pd
#generamos un dataframe para manejarlo con pandas
#empaquetando cada dato con su respectivo target
#cada target es un número del 0..3 que corresponde con el 
#indice de la categoría en el array train.target_names
a = pd.DataFrame(zip(train.data,train.target))
for category in train.target_names:
    print category, ":"
    category_id = train.target_names.index(category)
    print a.loc[a[1] == category_id].iloc[0,]




comp.graphics :
0    Subject: newss\nFrom: pollarda@physc1.byu.edu\...
1                                                    0
Name: 7, dtype: object
sci.med :
0    From: annick@cortex.physiol.su.oz.au (Annick A...
1                                                    1
Name: 5, dtype: object
soc.religion.christian :
0    From: Petch@gvg47.gvg.tek.com (Chuck Petch)\nS...
1                                                    2
Name: 2, dtype: object
talk.politics.guns :
0    From: tms@cs.umd.edu (Tom Swiss (not Swift, no...
1                                                    3
Name: 0, dtype: object


Revisar el módulo CountVectorizer para extraer caracterísitcas a partir de texto y explicar cómo funciona. Explicar qué es el modelo de bolsa de palabras.



Bolsa de palabras: Es un modelo de representación de documentos de texto. Estos se representan como sets de las palabras que lo conforman, sin tener en cuenta gramática u orden de palabras, manteniendo la multiplicidad de las mismas

CountVectorizer:A partir de un texto crea una estructura matricial, en la cual almacena el valor de la palabra y la cantidad de ocurrencias de la misma, al analizar una nueva palabra si la misma ya existia en la estructura aumenta la cantidad de ocurrencias, de lo contrario crea un nuevo registro. Esto se realiza por cada elemento del iterable de entrada. A continuación se muestra un ejemplo de la ejecución del CountVectorized y la generación del estimador:

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(binary=True)
content = ["hola mundo hola","hola hola hola otra"]
X = vectorizer.fit_transform(content)

Se imprimen las palabras que fueron tomadas como caracteristicas:

In [47]:
print vectorizer.get_feature_names()


[u'hola', u'mundo', u'otra']


Para cada caracteristica, se muestra la cantidad de repeticiones de cada palabra. Las filas corresponden a cada entrada de la lista "content":

In [48]:
print X.toarray()

[[1 1 0]
 [1 0 1]]


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
data = cv.fit_transform(list(a[0]))

Entrenar un clasificador bayesiano sencillo Multinomial. Explicar cada uno de los parámetros de este clasificador.

Se encuentran 3 parametros para la creación del clasificador Bayesiando Sencillo Multinomial:
- alpha
- fit_prior
- class_prior


Alpha:
Es el parámetro de suavizado.
Dada una instancia de entrenamiento, puede suceder que al momento de estimar la probabilidad condicionada a un target value 'y' para de una de las características 'c' del la instancia (en este caso son palabras dentro de un texto), si 'c' no figuró en ninguna instancia de entrenamiento con resultado 'y', la probabilidad calculada como la cantidad de ocurrencias de c con resultado 'y' en las instancias de entrenamiento sobre la cantidad total de instancias con resultado 'y', daría cero. El resultado nulo no es deseable ya que no es necesariamente representativo de la reaildad, posiblemente debido a que el conjunto de entrenamiento no fue lo suficientemente grande como para aproximar la probabilidad en cuestión. 

Por esto se re-define el cálculo de la probabilidad ya mencionada como:
P(c/y) = (cant(c/y) + aplha)/ (cant(y) + alpha*n)

Donde: 
- cant(c/y) es la cantidad de ocurrencias de la característica c en instancias de entrenamiendo con resultado 'y'
- cant(y) es la cantidad de instancias con resultado y
- n es la cantidad de características distintas
- alpha es un real entre 0 y 1

De esta forma, logramos obtener probabilidades pequeñas pero no nulas para todas las características que presentan la situación mencionada anteriormente.

fit_prior:
Es un parámetro booleano que cuando está en True, calcula por cada clasificación 'y', la probabilidad de que una instancia del conjunto de entrenamiento tomada al azar tenga clasificación 'y'. Cuando está seteada en False, asume una distribución uniforme.

class_prior:
Es un parámetro de tipo lista, que contiene las probabilidades para cada clasificación 'y'. Si el parámetro es epecificado, no se calculan las probabilidades de las clasificaciones en base al conjunto de entrenamiento.

In [13]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

El método fit(X,y,sample_weight) que corresponde al clasificador MultinomialNB, entrenará el clasificador en base al conjunto de entrenamiento X y sus respectivos atributo objetivo 'y'. Así también, sample_weigth aplica pesos podnerando a cada uno de los ejemplos individuales, si no se especifica se asume que no hay pesos especiales para los ejemplos. A continuación entrenamos el clasificador:

In [14]:
clf.fit(data.toarray(), train.target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Repetir el proceso pero utilizando el módulo TfidfTransformer en lugar de CountVectorizer. Explicar la medida Tfidf.

El factor TF es la suma de todas las ocurrencias ó el número de veces que aparece un término en un documento.
El factor IDF de un término es inversamente proporcional al número de documentos en los que aparece dicho término. Es el coeficiente que determina la capacidad discriminatoria del término de un documento con respecto a la colección. Es decir, una medida de si el término es común o no, en la colección de documentos.

La medida TF-TDF corresponde al peso de un término en un documento y se calcula como el producto de su frecuencia de aparición en dicho documento (TF) y su frecuencia inversa de documento (IDF).
Un peso alto en TF-IDF se alcanza con una elevada frecuencia del término (en el documento dado) y una pequeña frecuencia de ocurrencia del término en la colección completa de documentos. Cuando un término aparece en muchos documentos, ofrece un valor de TF-IDF cercano a 0.

In [33]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()

Se utiliza el siguiente vector de documentos para dar un ejemplo del funcionamiento de TfidfTransformer:

content = ["hola mundo hola","hola hola hola otra"]

Ese vector fue utilizado en partes anteriores, siendo transformado por medio del CountVectorizer como se muestra a continuación:

X = vectorizer.fit_transform(content)

Así, el siguiente código muestra el uso del TfidfTransformer:


In [34]:
dataTf = tfidf.fit_transform(X)
print vectorizer.get_feature_names()
print dataTf.toarray()

[u'hola', u'mundo', u'otra']
[[ 0.81818021  0.57496187  0.        ]
 [ 0.90554997  0.          0.42423963]]


Incluya los pasos de extracción de características y el clasificador dentro de un pipeline.

In [41]:
text_clf = Pipeline([('vect', vectorizer),
                     ('tfidf', tfidf),
                     ('clf', clf)])

parameters = {
    'vect__binary': (True, False), # If True, all non zero counts are set to 1.
    'tfidf__use_idf': (True,False),
    'tfidf__smooth_idf': (True,False),
    'clf__alpha': (0.001, 0.5, 1)
}

Incluir las métricas de precision, recall y fscore para cada cada una de las clases y la matriz de confusión probando contra el conjunto de testing.

In [42]:
from sklearn.metrics import precision_recall_fscore_support
text_clf.fit(train.data,train.target)

y_true = test.target
y_pred = text_clf.predict(test.data)

precision_recall_fscore_support(y_true, y_pred, average='macro')

(0.92313120047237662, 0.91224276033282514, 0.91250514740702227, None)

Incluya una GridSearch para ver cuales son los mejores hiper-parámetros del modelo

In [45]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(text_clf, parameters)
grid_search.fit(train.data, train.target)
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.986
Best parameters set:
	clf__alpha: 0.001
	tfidf__smooth_idf: True
	tfidf__use_idf: True
	vect__binary: True


Exlique cada uno de los parámetros de cada uno de los pasos del pipeline y pruebe con varios tipos de clasificadores de los vistos en el curso. ¿Con qué configuración de parámetros obtuvo mejores resultados? Aplique selección de atributos y valore los resultados obtenidos. Realice una prueba final contra el conjunto de pruebas.

Exlique cada uno de los parámetros de cada uno de los pasos del pipeline:

- clf alpha: es el parámetro utilizado durante el calculo de probabilidades. El mismo fue explicado anteriormente.

- tfidf use_idf: Cuando es True, utiliza 'inverse-document-frequency reweighting' y no únicamente term-frequency. Lo que implica que se obtiene más información midiendo la frecuencia de cada término en el total de documentos.

- tfidf smooth_idf: Simula agregar un documento que contiene cada término una vez. Esto se hace para prevenir divisiones entre cero.

- vect binary: No cuenta las repeticiones. Para aquellos valores del CountVectorized que den mayor que cero, los setea a 1, haciendo que la matiz de conteo se transforme en una matriz binaria, contando la aparición o no de la palabra.


Repita el ejercicio utilizando HashingVectorizer para extraer características. Explique las ventajas de este método de extracción de caraterísticas frente a CountVectorizer.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

La implementacion de hashVectorizer utiliza la funcion de hash para para encontrar el nombre de la cadena de tokens para la 
asignacion de indice de enteros.

Ventajas frente a CountVectorizer:
- Es muy baja la memoria escalable a los datasets grandes ya que no hay necesidad de almacenar un diccionario de vocabulario en la memoria
- Es muy rapido para serializar y deserializar ya que no tiene ningun estado, ademas de los parametros del constructor.
- Se puede utilizar en pipeline paralelo ya que no hay ningun estado calculado durante el ajuste.

Desventajas frente a CountVectorizer:
- No hay manera de calcular la transformacion inversa (de los indices de caracteistica a los nombres de la caracteristica 
de la secuencia) que puede ser un problema al intentar obtener que caracteristicas son las mas importantes a un modelo.
- Puede haber colisiones: los tokens distintos se pueden asignar al mismo indice de caracteristicas